In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
def NN2NA(NN):
  rows,columns=NN.shape

  columns_NA=np.count_nonzero(NN)
  if(rows != columns):
    print("Malformed NN Matrix. rows are not equal to columns")
    return none,none
  k=0
  NA=np.zeros((rows,columns_NA))
  arches = ["" for i in range(columns_NA)]

  for i in range(columns):
    for j in range(rows):
      if NN[i,j]==1:
        NA[i,k]=1
        NA[j,k]=-1
        if i==0:
          arches[k]="s->"+str(j+1)
        elif j==0:
          arches[k]=str(i+1)+"->s"
        elif i==columns-1:
          arches[k]="t->"+str(j+1)
        elif j==columns-1:
          arches[k]=str(i+1)+"->t"
        else:
          arches[k]=str(i+1)+"->"+str(j+1)
        k+=1
  return NA, arches


def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

In [3]:

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])


C = np.array([2, 2, 2, 5, 2, 1, 2])
Aeq, arc_idxs = NN2NA(NN)
beq = np.array([1, 0, 0, 0, 0, -1])
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n\n '
      'Arches in order:\n%s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s' % (C,arc_idxs,Aeq, beq, bounds))

## Optimizer inputs ## 
Cost vector: [2 2 2 5 2 1 2] 

 Arches in order:
['s->2', 's->3', '2->4', '2->t', '3->5', '4->t', '5->t'] 

A_eq Node-Arc matrix:
[[ 1.  1.  0.  0.  0.  0.  0.]
 [-1.  0.  1.  1.  0.  0.  0.]
 [ 0. -1.  0.  0.  1.  0.  0.]
 [ 0.  0. -1.  0.  0.  1.  0.]
 [ 0.  0.  0.  0. -1.  0.  1.]
 [ 0.  0.  0. -1.  0. -1. -1.]] 

b_eq demand-supply vector: [ 1  0  0  0  0 -1] 

Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None))


In [4]:
# OPTIMIZE:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

for name_method in 'interior-point', 'revised simplex':
    # OPTIMIZE:
    res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method=name_method)

    # GET THE SOLUTION:
    selarcs = get_selected_arcs(arc_idxs, res.x)
    print('\n ## Results for %s ## ' % name_method)
    print('The raw solution will be: %s' % res.x)
    print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
    print('The minimum cost will be: %0.2f\n' % res.fun)

print("\n\n")



 ## Results for interior-point ## 
The raw solution will be: [1.00 0.00 1.00 0.00 0.00 1.00 0.00]
The arcs that make the shortest path will be (from, to): ['s->2', '2->4', '4->t']
The minimum cost will be: 5.00


 ## Results for revised simplex ## 
The raw solution will be: [1.00 0.00 1.00 0.00 0.00 1.00 0.00]
The arcs that make the shortest path will be (from, to): ['s->2', '2->4', '4->t']
The minimum cost will be: 5.00






/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
